<a href="https://colab.research.google.com/github/luanps/pyserini/blob/master/Run_pyserini_ance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%%capture
!pip install pyserini

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [3]:
!pip install faiss-cpu

     |████████████████████████████████| 8.6 MB 10.0 MB/s 


## Dense retrieval with ANCE - MSMARCO Passage Ranking

In [ ]:
!python -m pyserini.dsearch --topics msmarco-passage-dev-subset \
                             --index msmarco-passage-ance-bf \
                             --encoded-queries ance-msmarco-passage-dev-subset \
                             --batch-size 36 \
                             --threads 12 \
                             --output run.msmarco-passage.ance.bf.tsv \
                             --output-format msmarco

In [ ]:
!gsutil cp run.* gs://luanps/information_retrieval/pyserini/ance/

### Evaluation

In [ ]:
#MRR Eval
!python -m pyserini.eval.msmarco_passage_eval msmarco-passage-dev-subset run.msmarco-passage.ance.bf.tsv >> msmarco_passage_mrr_eval.txt

In [ ]:
#TREC Eval
!python -m pyserini.eval.convert_msmarco_run_to_trec_run --input run.msmarco-passage.ance.bf.tsv \
                                                         --output run.msmarco-passage.ance.bf.trec

!python -m pyserini.eval.trec_eval -c -mrecall.1000 \
                                      -mmap msmarco-passage-dev-subset \
                                      run.msmarco-passage.ance.bf.trec >> msmarco_passage_eval_eval.txt

In [ ]:
!gsutil cp msmarco_passage_trec_eval.txt msmarco_passage_mrr_eval.txt gs://luanps/information_retrieval/pyserini/ance/

## Dense retrieval with ANCE - MSMARCO Document Ranking

In [ ]:
!python -m pyserini.dsearch --topics msmarco-doc-dev \
                             --index msmarco-doc-ance-maxp-bf \
                             --encoded-queries ance_maxp-msmarco-doc-dev \
                             --output run.msmarco-doc.passage.ance-maxp.txt \
                             --hits 1000 \
                             --max-passage \
                             --max-passage-hits 100 \
                             --output-format msmarco \
                             --batch-size 36 \
                             --threads 12

Using pre-defined topic order for msmarco-doc-dev
Attempting to initialize pre-encoded queries ance_maxp-msmarco-doc-dev.
query-embedding-ance_maxp-msmarco-doc-dev-20210419-9323ec.tar.gz: 14.2MB [00:00, 15.5MB/s]                
Extracting /root/.cache/pyserini/queries/query-embedding-ance_maxp-msmarco-doc-dev-20210419-9323ec.tar.gz into /root/.cache/pyserini/queries/query-embedding-ance_maxp-msmarco-doc-dev-20210419-9323ec.3d41ae797cb97e42649c4f4fa7b97d56...
Initializing ance_maxp-msmarco-doc-dev...
Attempting to initialize pre-built index msmarco-doc-ance-maxp-bf.
dindex-msmarco-doc-ance_maxp-bf-20210304-b2a1b0.tar.gz:   5% 2.63G/54.3G [01:50<41:27, 22.3MB/s]

In [ ]:
!gsutil cp run.* gs://luanps/information_retrieval/pyserini/ance/

### Evaluation

In [ ]:
#MRR Eval
! python -m pyserini.eval.msmarco_doc_eval --judgments msmarco-doc-dev --run run.msmarco-doc.passage.ance-maxp.txt >> msmarco_document_mrr_eval.txt

In [ ]:
#TREC Eval
! python -m pyserini.eval.convert_msmarco_run_to_trec_run --input/run.msmarco-doc.passage.ance-maxp.txt \
                                                          --output run.msmarco-doc.passage.ance-maxp.trec

! python -m pyserini.eval.trec_eval -c -mrecall.100 -mmap msmarco-doc-dev run.msmarco-doc.passage.ance-maxp.trec >> msmarco_document_trec_eval.txt

In [ ]:
!gsutil cp msmarco_document_trec_eval.txt msmarco_document_mrr_eval.txt gs://luanps/information_retrieval/pyserini/ance/